In [1]:
import sys
import torch as t
import torch
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
sys.path.append('/workspace/wilson/Finite-groups/src')
from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from model_utils import *
from group_utils import *
from jaxtyping import Float
from typing import Union
from einops import repeat
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
from llc import *
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
path = '/workspace/models/2024-08-14_21-24-30_gapS_5_'
models, params = load_models(path)

In [3]:
data = GroupData(params)

Intersection size: 14400/14400 (1.00)
Added 14400 elements from intersection
Added 0 elements from group 0: gapS(5)
Taking random subset: 5760/14400 (0.40)
Train set size: 5760/14400 (0.40)


In [4]:
loss_dict = test_loss(models[-1].to(device), data)

In [7]:
loss_dict['G0_loss'].argmin()

tensor(71, device='cuda:0')

In [35]:
model = models[-1][71].to(device)
lneurons, rneurons = model.get_neurons()
unembed = model.unembedding.data.detach()
lneurons, rneurons, unembed = lneurons.squeeze(0).to('cpu'), rneurons.squeeze(0).to('cpu'), unembed.squeeze(0).to('cpu')

In [36]:
irreps = data.groups[0].get_irreps()

In [37]:
irreps['4d-0'].shape

torch.Size([120, 4, 4])

In [38]:
irrep_bases = dict()
for name, irrep in irreps.items():
    irrep = einops.rearrange(irrep, 'N d1 d2 -> N (d1 d2)')
    irrep_bases[name] = torch.svd(irrep)[0]

In [44]:
lexpl = dict()
for name, basis in irrep_bases.items():
    proj = basis.T @ lneurons
    lexpl[name] = proj.norm(dim=0)**2 / lneurons.norm(dim=0)**2
    lexpl[name] = lexpl[name].nan_to_num(0.)

In [49]:
lexpl_total, lirrep = t.stack(list(lexpl.values()), dim=0).max(dim=0)

In [50]:
lexpl_total

tensor([0.0000e+00, 9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01,
        9.9999e-01, 9.9999e-01, 0.0000e+00, 0.0000e+00, 9.9999e-01, 0.0000e+00,
        9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01, 0.0000e+00, 9.9999e-01,
        9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01,
        9.9999e-01, 9.9999e-01, 9.9999e-01, 3.4028e+38, 9.9999e-01, 9.9999e-01,
        9.9999e-01, 9.9999e-01, 0.0000e+00, 9.9989e-01, 9.9999e-01, 9.9999e-01,
        9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01, 0.0000e+00,
        0.0000e+00, 9.9995e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01, 0.0000e+00,
        9.9999e-01, 9.9999e-01, 9.9999e-01, 0.0000e+00, 9.9999e-01, 9.9998e-01,
        9.9999e-01, 9.9998e-01, 9.9999e-01, 9.9999e-01, 0.0000e+00, 9.9998e-01,
        9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01,
        9.9997e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01,
        9.9999e-01, 0.0000e+00, 9.9999e-

In [52]:
lirrep[lexpl_total > 0.5]

tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 0, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [53]:
list(lexpl.keys())[5]

'4d-1'

In [54]:
irreps['4d-1']

tensor([[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.],
         [ 0.,  0.,  0.,  1.]],

        [[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.],
         [ 0., -1.,  1., -1.]],

        [[ 1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.],
         [ 0.,  0.,  1.,  0.],
         [ 0.,  1.,  0.,  0.]],

        ...,

        [[ 0.,  1., -1.,  1.],
         [ 0.,  0.,  0.,  1.],
         [ 0.,  0., -1.,  1.],
         [ 1.,  0., -1.,  0.]],

        [[ 0.,  0.,  0., -1.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  1., -1.,  0.],
         [ 1.,  0., -1.,  0.]],

        [[ 0.,  1., -1.,  1.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  1., -1.,  0.],
         [ 1.,  0., -1.,  0.]]])